In [2]:
import os, sys
import django
import pytz
from dateutil import parser
from django.utils.timezone import make_aware
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

sys.path.append('../..') # add path to project root dir
os.environ["DJANGO_SETTINGS_MODULE"] = "esr21.settings"

# Connect to Django ORM
django.setup()

  * Reading config from esr21.ini
Loading Data Encryption (init)...
 * loading keys from /Users/Ame/source/esr21/crypto_fields
 * loading rsa.restricted.public ... Done.
 * loading rsa.restricted.private ... Done.
 * loading rsa.local.public ... Done.
 * loading rsa.local.private ... Done.
 * loading aes.local ... Done.
 * loading aes.restricted ... Done.
 * loading salt.local ... Done.
 * loading salt.restricted ... Done.
 Done loading Data Encryption (init)...


/Users/Ame/.venvs/esr21/lib/python3.9/site-packages/simple_history/models.py:123: UserWarning: HistoricalRecords added to abstract model (HistoryManagerMixin) without inherit=True
  warnings.warn(msg, UserWarning)
/Users/Ame/.venvs/esr21/lib/python3.9/site-packages/simple_history/models.py:123: UserWarning: HistoricalRecords added to abstract model (ScheduleModelMixin) without inherit=True
  warnings.warn(msg, UserWarning)
/Users/Ame/.venvs/esr21/lib/python3.9/site-packages/simple_history/models.py:123: UserWarning: HistoricalRecords added to abstract model (VisitModelMixin) without inherit=True
  warnings.warn(msg, UserWarning)
/Users/Ame/.venvs/esr21/lib/python3.9/site-packages/simple_history/models.py:123: UserWarning: HistoricalRecords added to abstract model (CrfModelMixin) without inherit=True
  warnings.warn(msg, UserWarning)


Loading Data Encryption ...
 * found encryption keys in /Users/Ame/source/esr21/crypto_fields.
 * using model django_crypto_fields.crypt.
 Done loading Data Encryption.
Loading Edc Consent ...
 * checking for site consents ...
 * registered consents 'consents' from 'esr21_subject'
 * esr21_subject.informedconsent 1 covering 2021-05-01 UTC to 2022-05-31 UTC
 Done loading Edc Consent.
Loading Edc Identifier ...
 * identifier prefix: 999
 * check-digit modulus: 7
 Done loading Edc Identifier
Loading Edc Navbar ...
 * checking for site navbars ...
 * registered navbars 'navbars' from 'edc_calendar'
 * registered navbars 'navbars' from 'edc_consent'
 * registered navbars 'navbars' from 'edc_lab_dashboard'
 * registered navbars 'navbars' from 'edc_navbar'
 * registered navbars 'navbars' from 'edc_reference'
 * registered navbars 'navbars' from 'edc_visit_schedule'
 * registered navbars 'navbars' from 'esr21_follow'
 * registered navbars 'navbars' from 'esr21_export'
 * registered navbars 'na

/Users/Ame/.venvs/esr21/lib/python3.9/site-packages/django_q/conf.py:139: UserWarning: Retry and timeout are misconfigured. Set retry larger than timeout, 
        failure to do so will cause the tasks to be retriggered before completion. 
        See https://django-q.readthedocs.io/en/latest/configure.html#retry for details.
  warn(


Loading Edc Appointments ...
 * edc_appointment.appointment.
 Done loading Edc Appointments.
Loading Edc Lab ...
 * checking for labs ...
 Done loading Edc Lab.
Loading Edc Base ...
 * default TIME_ZONE Africa/Gaborone.
 Done loading Edc Base.
Loading Edc Facility ...
 * 7-Day Clinic MO(100 slots), TU(100 slots), WE(100 slots), TH(100 slots), FR(100 slots), SA(100 slots), SU(100 slots).
 * 5-Day Clinic MO(100 slots), TU(100 slots), WE(100 slots), TH(100 slots), FR(100 slots).
 Done loading Edc Facility.
Loading Edc Metadata ...
 * using default metadata models from 'edc_metadata'
 Done loading Edc Metadata.
Loading Edc Protocol ...
 * BHP150: BHP150 | ADZ 1222 - ESR-21-21311.
 * Study opening date: 2021-04-01 UTC
 * Expected study closing date: 2025-12-01 UTC
 Done loading Edc Protocol.
Loading Edc Visit Tracking ...
 * esr21_subject.subjectvisit uses model attr 'subject_visit'
 Done loading Edc Visit Tracking.
Loading Edc Timepoint ...
 * 'edc_appointment.appointment' is a timepoint m

In [83]:
import pandas as pd
from django_pandas.io import read_frame
from django.db.models import Count, Q, F
from esr21_subject.models import EligibilityConfirmation, ScreeningEligibility, RapidHIVTesting
from edc_constants.constants import YES

# Table 2
# Participants exclusion criterion
qs = EligibilityConfirmation.objects.values('site_id').annotate(
    age_lt_18=Count('site_id', filter=Q(age_in_years__lt=18)),
    vaccine_other_than_influenza=Count('site_id', filter=Q(received_vaccines=YES)),
    other_studies=Count('site_id', filter=Q(participating_in_other_studies=YES)))
df = read_frame(qs, fieldnames=['age_lt_18', 'vaccine_other_than_influenza', 'other_studies'], index_col='site_id')

qs1 = ScreeningEligibility.objects.filter(created__date__lte=end_dt.date()).values('site_id').annotate(
    hypersensitive_active_substance=Count('site_id', filter=Q(substance_hypersensitivity=YES)),
    pregnancy_status=Count('site_id', filter=Q(pregnancy_status=YES)),
    thrombosis_or_thrombocytopenia=Count('site_id', filter=Q(thrombosis_or_thrombocytopenia=YES)),
    history_of_guillain_barre=Count('site_id', filter=Q(guillain_barre_syndrome=YES)),
    suspected_immuno_condition=Count('site_id', filter=Q(suspected_immuno_condition=YES)),
    clinical_bleeding=Count('site_id', filter=Q(clinical_bleeding=YES)),
    covid_symptoms=Count('site_id', filter=Q(covid_symptoms__isnull=False)))
df1 = read_frame(qs1, fieldnames=['hypersensitive_active_substance', 'pregnancy_status', 'thrombosis_or_thrombocytopenia',
                                  'history_of_guillain_barre', 'suspected_immuno_condition', 'clinical_bleeding', 'covid_symptoms'], index_col='site_id')
frames = [df, df1]
exclusion_df = pd.concat(frames)

grouped_df = exclusion_df.groupby('site_id')
grouped = grouped_df.first()
swapped_df = grouped.T
swapped_df.rename(columns={'esr21': 'Gaborone',
                           'esr21_41': 'Maun',
                           'esr21_42': 'Serowe',
                           'esr21_43': 'Fracistown',
                           'esr21_44': 'Selibe Phikwe'}, inplace=True)
swapped_df

site_id,Gaborone,Maun,Serowe,Fracistown,Selibe Phikwe
age_lt_18,0.0,0.0,0.0,0.0,0.0
vaccine_other_than_influenza,0.0,0.0,0.0,1.0,2.0
other_studies,6.0,0.0,1.0,1.0,1.0
hypersensitive_active_substance,1.0,0.0,0.0,2.0,2.0
pregnancy_status,1.0,0.0,2.0,0.0,0.0
thrombosis_or_thrombocytopenia,0.0,0.0,0.0,0.0,0.0
history_of_guillain_barre,1.0,0.0,3.0,0.0,0.0
suspected_immuno_condition,0.0,1.0,0.0,0.0,0.0
clinical_bleeding,0.0,0.0,0.0,0.0,0.0
covid_symptoms,31.0,1.0,6.0,0.0,0.0


In [5]:
import pandas as pd
from django_pandas.io import read_frame
from django.db.models import Count, Q, F
from esr21_subject.models import RapidHIVTesting, InformedConsent
from edc_constants.constants import POS, NEG, IND, MALE, FEMALE, YES

# Table 4
# HIV Screening

male_participants = InformedConsent.objects.filter(gender=MALE).values_list('subject_identifier')
female_participants = InformedConsent.objects.filter(gender=FEMALE).values_list('subject_identifier')

qs = RapidHIVTesting.objects.filter(rapid_test_done=YES).values('site_id').annotate(
    participants_tested=Count('site_id'),
    hiv_pos=Count('site_id', filter=Q(rapid_test_result=POS)),
    hiv_pos_male=Count('site_id', filter=(Q(rapid_test_result=POS) & Q(subject_visit__subject_identifier__in=male_participants))),
    hiv_pos_female=Count('site_id', filter=(Q(hiv_result=POS) & Q(subject_visit__subject_identifier__in=female_participants))),
    hiv_neg=Count('site_id', filter=Q(hiv_result=NEG)),
    hiv_neg_male=Count('site_id', filter=(Q(hiv_result=NEG) & Q(subject_visit__subject_identifier__in=male_participants))),
    hiv_neg_female=Count('site_id', filter=(Q(hiv_result=NEG) & Q(subject_visit__subject_identifier__in=female_participants))),
    indeterminate=Count('site_id', filter=Q(hiv_result=IND)))
df = read_frame(qs, fieldnames=['participants_tested', 'hiv_pos', 'hiv_pos_male', 'hiv_pos_female',
                                'hiv_neg', 'hiv_neg_male', 'hiv_neg_female', 'indeterminate'], index_col='site_id')
swapped_df = df.T
swapped_df.rename(columns={'esr21': 'Gaborone',
                           'esr21_41': 'Maun',
                           'esr21_42': 'Serowe',
                           'esr21_43': 'Fracistown',
                           'esr21_44': 'Selibe Phikwe'}, inplace=True)
swapped_df

site_id,Gaborone,Maun,Serowe,Fracistown,Selibe Phikwe
participants_tested,2595,1119,901,976,660
hiv_pos,589,288,226,218,114
hiv_pos_male,195,134,130,87,61
hiv_pos_female,394,154,96,131,53
hiv_neg,1455,664,417,434,450
hiv_neg_male,669,366,279,269,287
hiv_neg_female,786,298,138,165,163
indeterminate,0,1,0,0,0


In [128]:
from django_pandas.io import read_frame
from django.db.models import Count, Q, F
from django.db.models.functions.datetime import TruncMonth
from edc_registration.models import RegisteredSubject

# Table 6
# Monthly, enrolments per site

qs = RegisteredSubject.objects.annotate(month=TruncMonth('created')).values('site_id', 'month').annotate(
    total_entries=Count('site_id'))
df = read_frame(qs, fieldnames=['month', 'total_entries'], index_col='site_id')
df
grouped = df.groupby(['site_id', df['month'].dt.strftime('%B')])['total_entries']
grouped.first()

site_id   month    
esr21     December      539
          November     1586
          October       783
          September     381
esr21_41  December      182
          November      758
          October       187
esr21_42  December      196
          November      521
          October       198
esr21_43  December      106
          November      772
          October       158
esr21_44  December       84
          November      536
          October        73
Name: total_entries, dtype: int64

In [149]:
from django_pandas.io import read_frame
from django.db.models import FloatField
from django.db.models import Count, Q, F
from django.db.models.functions import Cast
from esr21_subject.models import AdverseEventRecord

# Table 8
# Adverse symptoms stats
overall_total = AdverseEventRecord.objects.count()

qs = AdverseEventRecord.objects.values('pt_name').annotate(
    total=Count('id'),
    percentage=Cast((F('total')/overall_total*100.0), FloatField()))
df = read_frame(qs, fieldnames=['pt_name', 'total', 'percentage'])
df

,pt_name,total,percentage
0,Decreased Appetite,3,0.958466
1,headaches,7,2.236422
2,Fatigue,38,12.140575
3,PYREXIA,21,6.709265
4,Vomiting,5,1.597444
5,chest pain,3,0.958466
6,Arthralgia,4,1.277955
7,Headache,68,21.725240
8,Myalgia,7,2.236422
9,,22,7.028754
